## Import Repository

In [1]:
!git clone https://github.com/federico2879/MLDL2024_semantic_segmentation.git

Cloning into 'MLDL2024_semantic_segmentation'...
remote: Enumerating objects: 552, done.
remote: Counting objects: 100% (203/203), done.
remote: Compressing objects: 100% (108/108), done.
remote: Total 552 (delta 117), reused 150 (delta 93), pack-reused 349
Receiving objects: 100% (552/552), 272.10 KiB | 3.36 MiB/s, done.
Resolving deltas: 100% (324/324), done.


## Import package

In [2]:
!pip install -U fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 723.8 kB/s eta 0:00:00 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=7cc43d569d6d323c860f5266c870b4e3837f6ee40b37b99bdd5e7ad60c35afb4
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31532 sha256=89b375f194de1d61ca12f71397f0472c6b56ae28c73cd0ebd4678c23b03fc2b2
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built fvcore iopath


In [3]:
import torch
from torch import nn
import torchvision
import torchvision.transforms as transforms
from torchvision.transforms.functional import InterpolationMode
from torch.utils.data import DataLoader
import numpy as np
from MLDL2024_semantic_segmentation.datasets.cityscapes import CityScapes
from MLDL2024_semantic_segmentation.models.deeplabv2.deeplabv2 import *
from MLDL2024_semantic_segmentation.train import * 
from MLDL2024_semantic_segmentation.utils import *
from MLDL2024_semantic_segmentation.models.metrics import * 
from MLDL2024_semantic_segmentation.models.IOU import * 

## Setup

In [4]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

# Setup fixed parameters
num_classes = 19
num_epochs = 50
batch_size = 4

cuda


## Dataset

In [5]:
# Transformations
transform_image = transforms.Compose([
    transforms.Resize((1024, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

transform_target = transforms.Compose([
    transforms.Resize((1024, 512), interpolation=InterpolationMode.NEAREST)
])

In [6]:
# Create datasets and dataloader

train_dataset = CityScapes('/kaggle/input/cityscapes/Cityscapes/Cityspaces', 
                           split = 'train', transform = transform_image, 
                           label_transform = transform_target)

dataloader_train = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = CityScapes('/kaggle/input/cityscapes/Cityscapes/Cityspaces', 
                         split = 'val', transform = transform_image, 
                         label_transform = transform_target)

dataloader_val = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

## Network, Loss, Optimizer

In [7]:
# Pretrained
pretrained = torchvision.models.resnet101(pretrained=True).to(device)
torch.save(pretrained.state_dict(), "DeepLab_resnet_pretrained_imagenet.pth")

# Inizialization of the model
model = get_deeplab_v2().to(device)
diz_optim = model.optim_parameters(0.001) # parameters + learning rates

# Putting on the 2 gpus
model = torch.nn.DataParallel(model, device_ids = [0,1]).to(device)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:01<00:00, 150MB/s]  


Deeplab pretraining loading...


In [8]:
# Define loss and optimizer
loss_fn = nn.CrossEntropyLoss(ignore_index=255)
optimizer = torch.optim.SGD(diz_optim, momentum=0.9, weight_decay=0.0005)

/opt/conda/lib/python3.10/site-packages/torch/_compile.py:24: UserWarning: optimizer contains a parameter group with duplicate parameters; in future, this will cause an error; see github.com/pytorch/pytorch/issues/40967 for more information
  return torch._dynamo.disable(fn, recursive)(*args, **kwargs)


## Training

In [9]:
# metrics

meanIOU_tr = np.zeros((num_epochs,1))
IOU_tr = np.zeros((num_epochs,num_classes))
loss_tr = np.zeros((num_epochs,1))


meanIOU_val = np.zeros((num_epochs,1))
IOU_val = np.zeros((num_epochs,num_classes))
loss_val = np.zeros((num_epochs,1))

In [10]:
# Set the random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [11]:
for epoch in range(num_epochs):
    lr_1x = poly_lr_scheduler(optimizer, 0.001, epoch, lr_decay_iter=1,
                      max_iter=num_epochs, power=0.9)
    optimizer.param_groups[1]['lr'] = 10*lr_1x
    meanIOU_tr[epoch], IOU_tr[epoch,:], loss_tr[epoch] = train(model, optimizer, dataloader_train, loss_fn, num_classes)
    meanIOU_val[epoch], IOU_val[epoch,:], loss_val[epoch] = test(model, dataloader_val, loss_fn, num_classes)
    print(f"epoch: {epoch + 1}, Validation IOU: {meanIOU_val[epoch,0]:.2f}")

    torch.save({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'meanIOU_tr': meanIOU_tr,
        'IOU_tr': IOU_tr,
        'loss_tr': loss_tr,
        'meanIOU_val': meanIOU_val,
        'IOU_val': IOU_val,
        'loss_val': loss_val
        },"checkpoint.pth.tar")

epoch: 1, Validation IOU: 0.13

epoch: 2, Validation IOU: 0.15

epoch: 3, Validation IOU: 0.18

epoch: 4, Validation IOU: 0.19

epoch: 5, Validation IOU: 0.19

epoch: 6, Validation IOU: 0.22

epoch: 7, Validation IOU: 0.22

epoch: 8, Validation IOU: 0.23

epoch: 9, Validation IOU: 0.25

epoch: 10, Validation IOU: 0.26

epoch: 11, Validation IOU: 0.26

epoch: 12, Validation IOU: 0.27

epoch: 13, Validation IOU: 0.26

epoch: 14, Validation IOU: 0.28

epoch: 15, Validation IOU: 0.28

epoch: 16, Validation IOU: 0.28

epoch: 17, Validation IOU: 0.30

epoch: 18, Validation IOU: 0.29

epoch: 19, Validation IOU: 0.29

epoch: 20, Validation IOU: 0.31

epoch: 21, Validation IOU: 0.29

epoch: 22, Validation IOU: 0.30

epoch: 23, Validation IOU: 0.32

epoch: 24, Validation IOU: 0.33

epoch: 25, Validation IOU: 0.30

epoch: 26, Validation IOU: 0.33

epoch: 27, Validation IOU: 0.33

epoch: 28, Validation IOU: 0.33

epoch: 29, Validation IOU: 0.34

epoch: 30, Validation IOU: 0.34

epoch: 31, Validati

In [11]:
from MLDL2024_semantic_segmentation.load_checkpoint import *

model, optimizer, start_epoch, meanIOU_tr, IOU_tr, loss_tr, meanIOU_val, IOU_val, loss_val \
= load_checkpoint(model, optimizer,"/kaggle/input/deeplab-s2-1p/checkpoint.pth.tar")

for epoch in range(start_epoch,num_epochs):
    lr_1x = poly_lr_scheduler(optimizer, 0.001, epoch, lr_decay_iter=1,
                      max_iter=num_epochs, power=0.9)
    optimizer.param_groups[1]['lr'] = 10*lr_1x
    meanIOU_tr[epoch], IOU_tr[epoch,:], loss_tr[epoch] = train(model, optimizer, dataloader_train, loss_fn, num_classes)
    meanIOU_val[epoch], IOU_val[epoch,:], loss_val[epoch] = test(model, dataloader_val, loss_fn, num_classes)
    print(f"epoch: {epoch + 1}, Validation IOU: {meanIOU_val[epoch,0]:.2f}")

    torch.save({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'meanIOU_tr': meanIOU_tr,
        'IOU_tr': IOU_tr,
        'loss_tr': loss_tr,
        'meanIOU_val': meanIOU_val,
        'IOU_val': IOU_val,
        'loss_val': loss_val
        },"checkpoint.pth.tar")


epoch: 34, Validation IOU: 0.34
epoch: 35, Validation IOU: 0.35
epoch: 36, Validation IOU: 0.36
epoch: 37, Validation IOU: 0.35
epoch: 38, Validation IOU: 0.36
epoch: 39, Validation IOU: 0.35
epoch: 40, Validation IOU: 0.34
epoch: 41, Validation IOU: 0.36
epoch: 42, Validation IOU: 0.37
epoch: 43, Validation IOU: 0.36
epoch: 44, Validation IOU: 0.36
epoch: 45, Validation IOU: 0.35
epoch: 46, Validation IOU: 0.36
epoch: 47, Validation IOU: 0.35
epoch: 48, Validation IOU: 0.36
epoch: 49, Validation IOU: 0.35
epoch: 50, Validation IOU: 0.36


In [14]:
# final print

print(f"Final mIOU: {meanIOU_val[epoch,0]:.2f}")
print("Final IOU classes")
print(IOU_val[epoch,:])

flops = Flops(model, 1024, 512, device)

latency = Latency_FPS(model, 1024, 512, device)

print(f"Latency: {latency}")

Final mIOU: 0.36
Final IOU classes
[0.93854812 0.61444264 0.76553052 0.1674058  0.10121503 0.11950016
 0.06397499 0.19969204 0.80061446 0.44759565 0.7482047  0.33163708
 0.0802935  0.75216987 0.11348163 0.17503018 0.0279584  0.0653875
 0.33121389]
| module                         | #parameters or shape   | #flops     |
|:-------------------------------|:-----------------------|:-----------|
| module                         | 43.901M                | 0.375T     |
|  conv1                         |  9.408K                |  1.233G    |
|   conv1.weight                 |   (64, 3, 7, 7)        |            |
|  bn1                           |  0.128K                |  16.777M   |
|   bn1.weight                   |   (64,)                |            |
|   bn1.bias                     |   (64,)                |            |
|  layer1                        |  0.216M                |  7.155G    |
|   layer1.0                     |   75.008K              |   2.487G   |
|    layer1.0.conv1   

In [13]:
# writing csv
import csv

with open('meanIOU_tr.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(meanIOU_tr)

with open('IOU_tr.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(IOU_tr)

with open('loss_tr.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(loss_tr)

with open('meanIOU_val.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(meanIOU_val)

with open('IOU_val.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(IOU_val)

with open('loss_val.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(loss_val)